<div class="alert block alert-info alert">

# <center> Scientific Programming in Python

## <center>Karl N. Kirschner<br>Bonn-Rhein-Sieg University of Applied Sciences<br>Sankt Augustin, Germany

# <center> Testing Inside your Code<br><br>and<br><br>Testing the Code Itself

<!-- <br><br> -->

<hr style="border:2px solid gray"></hr>

**Note**: All user-defined functions in the notebook do not include document strings (i.e. block comments) or internal checks. This is purposely done to focus on the teaching aspects of the lecture. **A full and proper user-defined function would include these.**

# Context: The Hidden Cost of Untested Code


**The "I Know It Works" Illusion**
- Testing code mentally or with a few temporary `print()` statements.
    - Works **today** and for this **one case**.
    - But tomorrow may be different: a colleague changes a dependency, or the code is refactored
        - That small, manual `print()` check is lost.

**Take-Home Message**
- Untested code is problematic.
    - Reduced confidence in it
    - Results in fear of making changes
    - Costly debugging

<br>

- Testing flips this dynamic
    - Gives you confidence to refactor, upgrade, and scale your code
        - If you break something, the tests will catch it
    - Debugging is faster and easier

# Unit Tests

- An 'automated' procedure to verify a **small**, **isolated** code piece (i.e., a **unit**).
    - Make sure that the unit works as intended under different conditions.
 
## Unit Tests Characteristics

- <font color='DodgerBlue'>**Isolation**</font>
    - Focuses on the smallest testable part of the code (e.g., single function).
    - It must be run in isolation (i.e., doesn't rely on external factors like databases).

- <font color='DodgerBlue'>**Assertion / Raise Statements**</font>
    - Every unit test contains one or more assertions/raise statements.
        - An assertion is a conditional check (e.g., `assert calculated_value == expected_value`).
        - If it fails, the test stops and reports an error.

- <font color='DodgerBlue'>Automation</font>
    - The test is code itself.
    - It can be run instantly and repeatedly by a testing framework (e.g., pytest, unittest).
    - This is crucial for speed and consistency.

- <font color='DodgerBlue'>Speed</font>
    - Unit tests must execute very quickly (milliseconds).
        - They will be run constantly by developers (or automated build systems).

### Assert vs. Raise

- Comparing <font color='DodgerBlue'>two floats</font> to be within a <font color='DodgerBlue'>tolerance</font> range

In [1]:
tolerance = 0.001

calculated_number = 0.115
expected_number = 0.117

#### Standard `assert`: raises an AssertionError

Condition
- `abs(calculated_number - expected_number)` must be **less than** the `tolerance` value
- If `True`, then continue.
- If `False`, then **raise error**:

In [2]:
abs(calculated_number - expected_number) <= tolerance

False

In [3]:
assert abs(calculated_number - expected_number) <= tolerance, "Values are too far apart."

AssertionError: Values are too far apart.

#### Custom Raise: Checks the condition, explicitly raises an exception on failure

- `abs(calculated_number - expected_number)` is **greater than** the `tolerance` value
- If `True`, then **raise error**:
- If `False`, then continue.

In [4]:
abs(calculated_number - expected_number) >= tolerance

True

In [5]:
if abs(calculated_number - expected_number) >= tolerance:
    raise AssertionError(f"Error: Expected value was {expected_number:.3f}, but obtained {calculated_number:.3f}.")

AssertionError: Error: Expected value was 0.117, but obtained 0.115.

## Example

In [6]:
def calculate_grade(score, max_score=100):
    """ Calculates the percentage grade from a score. """
    
    percentage = (score / max_score) * 100

    return percentage

**Manual check**

In [7]:
print(calculate_grade(80))

80.0


Look correct - great!

## Unit Test

In [8]:
def test_calculate_grade():
    """ Verifies the function's output for known good data. """

    # 1. Standard case check (80/100 should be 80.0)
    assert calculate_grade(80) == 80.0

    # 2. Edge case check (max_score is not 100)
    assert calculate_grade(25, max_score=50) == 50.0

    print("Test Passed.")

In [9]:
test_calculate_grade()

Test Passed.


In [10]:
We will add a `try`-`except` statement

SyntaxError: invalid syntax (2891776488.py, line 1)

In [11]:
try:
    test_calculate_grade()
except AssertionError:
    print("TEST FAILED! The test caught a bug that the simple manual check missed.")
    print("This is the power of a Unit Test: immediate feedback and regression prevention.")

Test Passed.


Now, let's say in 10 months a colleague moves the 100 to an integer 100 global position.

In [12]:
def calculate_grade(score, max_score=100):
    """ Buggy - calculates the percentage grade from a score. """
    
    # BUG: Forgot to multiply by 100
    percentage = (score / max_score) 
    
    return percentage

In [13]:
try:
    test_calculate_grade()
except AssertionError:
    print("TEST FAILED! The test caught a bug that the original/earlier simple manual check missed.")
    print("This is the power of a Unit Test: immediate feedback and regression prevention.")

TEST FAILED! The test caught a bug that the original/earlier simple manual check missed.
This is the power of a Unit Test: immediate feedback and regression prevention.


---

# Exceptions: testing for expected conditions
<!-- EAFP ("<font color='DodgerBlue'>**E**asier to **A**sk for **F**orgiveness than **P**ermission</font>")
- often adopted by programmers,
- but is that good practice?

EAFP can be implemented via the: -->
### `try`-`except` statement
- tells your code to try something
- then tell it what to do if it fails based on an exception type

 
#### Strengths:
1. your code will <font color='DodgerBlue'>**continue**</font> when it encounters a problem<br><br>

2. <font color='DodgerBlue'>**faster** than if statements</font> for when <font color='DodgerBlue'>majority of the planned tasks are **expected** to be **successful**</font> (i.e., they don't encounter an exception)

<br><br>
**Simple Example**

In [ ]:
try:
    print(5/0)
except ZeroDivisionError:
    print("Error: You can't have a zero in the denominator.")

**A More Sophisticated Example**

Let's set up a <font color='DodgerBlue'>division calculator</font> that allows users to input numbers and quit at any time using while and if loops (to demonstrate via a comparison of code).

1. First, set something up without `try`-`except` in order to see its advantage later.
2. Second, do the same thing with `try`-`except`

<font color='DodgerBlue'>Demonstrate the following</font>:
1. normal operation
2. exiting by typing 'q'
3. <font color='Red'>O</font> (i.e., a capital alphabet letter "O", as in "O"ktoberfest (also demonstrates traceback error))

Without `try`-`except` statement:

In [ ]:
## print('Type two numbers that you want to be divided.')
print("Type 'q' to quit.")
print()

while True:
    numerator = input('Numerator = ')
    if numerator == 'q':
        break

    denominator = input('Denominator = ')
    if denominator == 'q':
        break

    if denominator == '0':
        print("You can't have a zero in the denominator.")
        break

    answer = float(numerator)/float(denominator)
    print(f'Answer for {numerator}/{denominator} = {answer}\n')

Modify the above code to use a **`try`-`except` statement**, and try it with <font color='Red'>O</font>.

**Note**: <font color='DodgerBlue'>Multiple `except` conditions via a **tuple**</font>:<br>
`except (ZeroDivisionError, ValueError):`
- `ZeroDivisionError` when the denominator is zero
- `ValueError` for when a string is given as an input

In [ ]:
print('Type two numbers that you want to be divided.')
print("Type 'q' to quit.")
print()

while True:
    numerator = input('Numerator = ')
    if numerator == 'q':
        break

    denominator = input('Denominator = ')
    if denominator == 'q':
        break

    try:
        answer = float(numerator)/float(denominator)
        print(f'Answer for {numerator}/{denominator} = {answer}\n')

    except (ZeroDivisionError, ValueError):
        print('Your input was either not a number, or you are dividing by a zero.')

<font color='DodgerBlue'>Now the code continues to run, even though an error was raised!</font>

<hr style="border:2px solid gray"></hr>

<h1 align='center'>Test Driven Development
    
<h2 align='center'> a.k.a. Unit Tests

https://docs.python.org/3/library/unittest.html

## Test Driven Development - writing tests before you write your production code
1. Ensures proper and directed functionality of your code
 - creating **concise** code that does a **single** thing (e.g., user-defined functions)
2. Helps you plan your code - what do you **actually want** to do (critical thinking)
3. Reduces **errors**
4. Ensures **reproducibility**
5. Helps to ensure a code's **long life**

## The Workflow Concept
1. Write a failing test
2. Run and ensure failure
3. Write code to pass
4. Run and ensure passing
5. Refactor (i.e., restructure/clean up code without changing it final result)
6. Redo steps 1-5

## Scientific and Data Research
It is **CRITICAL** that:
1. you get the correct results
2. you make it generate reproducible results, especially as the code becomes bigger (and changes)

#### Assert statements that can be used in unittest library

https://docs.python.org/3/library/unittest.html#module-unittest


| Method | Checks that | New in |
|:------|:-:|:-:|
| assertEqual(a, b) | a == b | |
| assertNotEqual(a, b)| a != b | |
| assertTrue(x) | bool(x) is True | |
| assertFalse(x) | bool(x) is False | |
| assertIs(a, b) | a is b | 3.1 |
| assertIsNot(a, b) | a is not b | 3.1 |
| assertIsNone(x) | x is None | 3.1 |
| assertIsNotNone(x) | x is not None | 3.1 |
| assertIn(a, b) | a in b | 3.1 |
| assertNotIn(a, b) | a not in b | 3.1 |
| assertIsInstance(a, b) | isinstance(a, b) | 3.2 |
| assertNotIsInstance(a, b) | not isinstance(a, b) | 3.2 |
| | | |
| | | |
| assertAlmostEqual(a, b) | round(a-b, 7) == 0 | |
| assertNotAlmostEqual(a, b) | round(a-b, 7) != 0 | |
| assertGreater(a, b) | a > b | 3.1 |
| assertGreaterEqual(a, b) | a >= b | 3.1 |
| assertLess(a, b) | a < b | 3.1 |
| assertLessEqual(a, b) | a <= b | 3.1 |
| assertRegex(s, r) | r.search(s) | 3.1 |
| assertNotRegex(s, r) | not r.search(s) | 3.2 |
| assertCountEqual(a, b) | a and b have the same elements in the same number, regardless of their order. | 3.2 |


| Method | Used to compare | New in|
|:------|:-:|:-:|
| assertMultiLineEqual(a, b) | strings | 3.1 |
| assertSequenceEqual(a, b) | sequences | 3.1 |
| assertListEqual(a, b) | lists | 3.1 |
| assertTupleEqual(a, b) | tuples | 3.1 |
| assertSetEqual(a, b) | sets or frozensets | 3.1 |
| assertDictEqual(a, b) | dicts | 3.1 |

Demonstrate the following two scenarios:
1. Scenario 1: the unit test runs with everything correct
2. Scenario 2: the unit test runs, but with errors
     - A new `assertEqual` is added

**Note**: We will include <font color='DodgerBlue'>additional assert statements just to demonstrate how the output of a unit test looks like</font>, even though it is not relevant to our user-defined function.

**Scenario 1**: the unit test runs with everything <font color='DodgerBlue'>correctly</font>

Define a user-defined function to demo how that is done:

In [ ]:
def hello_world():
    return 'hello world'

**Note**: `isupper()`

- `str.isupper()`: Return `True` if all characters in a given string are uppercase, otherwise it is `False`.
    
    - https://docs.python.org/3/library/stdtypes.html

In [ ]:
import unittest


class MyFirstUniTTests(unittest.TestCase):

    def test_isEqual(self):
        self.assertEqual(hello_world(), 'hello world')

    def test_isLess(self):
        self.assertLess(5, 10)

    def test_isLessEqual(self):
        self.assertLessEqual(10, 10)

    def test_isUpperTrue(self):
        self.assertTrue('FOO'.isupper())

    def test_isUpperFalse(self):
        self.assertFalse('Foo'.isupper())


## Normal usage (in a .py script)
#if __name__ == '__main__':
#    unittest.main()

## For usage in jupyter and colaboratory (due to the kernel)
if __name__ == '__main__':
    unittest.main(argv=['ignored', '-v'], exit=False)

**Scenario 2**: the unit test runs, but some <font color='red'>errors</font> occur

In [ ]:
class MyFirstUniTTests(unittest.TestCase):

    def test_fail(self):
       self.assertEqual(hello_world(), 'bye world')

    def test_isEqual(self):
        self.assertEqual(hello_world(), 'hello world')

    def test_isLess(self):
        self.assertLess(5, 10)

    def test_isLessEqual(self):
        self.assertLessEqual(10, 10)

    def test_isUpperTrue(self):
        self.assertTrue('FOO'.isupper())

    def test_isUpperFalse(self):
        self.assertFalse('Foo'.isupper())


if __name__ == '__main__':
    unittest.main(argv=['ignored', '-v'], exit=False)

## PyTest

1. A command-line (e.g., using a bash shell) driven testing approach
2. Simplifies and helps organize unit tests
    - done by creating **user-defined functions** for **each test** that you want to do

https://docs.pytest.org/en/7.1.x/contents.html#

In [ ]:
%%writefile test_sum.py
''' The following will be created:
        1. Four unit test functions
            a. First 3 will pass
            b. Last 1 will fail
'''

def test_pass_add_list_1():
    ''' 1st unit test
    '''
    test_list = [1, 2, 3, 4]
    assert sum(test_list) == 10


def test_pass_add_list_2():
    ''' 2nd unit test
    '''
    test_list = [1, 2, 3, 4, 5]
    assert sum(test_list) == 15


def test_pass_add_list_3():
    ''' 3rd unit test
    '''
    test_list = [1, 2, 3, 4, 5, 6]
    assert sum(test_list) == 21


def test_fail_add_list_4():
    ''' 4th unit test
        Should Fail
    '''
    print('PRINT STATEMENT FOR FAILING TEST FUNCTION.')
    
    test_list = [1, 2, 3, 4, 5, 6]
    assert sum(test_list) == 0

PyTest will give the following output:
- `.` (dot) = test <font color='DodgerBlue'>passed</font>
- `F` =  test has <font color='DodgerBlue'>failed</font>
- `E` =  test raised an <font color='DodgerBlue'>unexpected exception</font>

In [ ]:
! pytest test_sum.py

**Output**
1. The first three test passed
2. The fourth test fails
3. A traceback is given concerning the error
4. None of the print statements are seen

<font color='DodgerBlue'>To see print commands</font> within the user-defined functions, <font color='DodgerBlue'>use `-s` option</font>:

(`-s` is a shortcut for `--capture=no` - see `pytest --help`)

In [ ]:
! pytest test_sum.py -s

Clean up directory:

In [ ]:
%rm test_sum.py

## Example of Notebook function and PyTesting

Create a User-defined Function

In [ ]:
def calculate_area(length, width):
    """ Calculate the area of a rectangle.
    """
    if length <= 0 or width <= 0:
        raise ValueError("Dimensions must be positive.")
    return length * width

Create PyTest

In [ ]:
%%writefile test_calculate_area.py
import pytest

def calculate_area(length, width):
    """ Calculate the area of a rectangle.
    """
    if length <= 0 or width <= 0:
        raise ValueError("Dimensions must be positive.")
    return length * width


def test_positive_dimensions():
    # Test case with standard positive inputs
    assert calculate_area(5, 4) == 20


def test_square_dimensions():
    # Test case for a square (equal sides)
    assert calculate_area(10, 10) == 100


def test_zero_or_negative_dimensions():
    # Test case for invalid input using pytest.raises
    with pytest.raises(ValueError):
        calculate_area(-1, 5)
    with pytest.raises(ValueError):
        calculate_area(0, 5)

In [ ]:
! pytest test_calculate_area.py

In [ ]:
%rm test_calculate_area.py

## Introducing `pytest.mark.parametrize`

In [ ]:
%%writefile test_even.py
import pytest


def add(number_1: int|float, number_2: int|float) -> float:
    """
    Returns the sum of two numbers.

    Parameters:
        number_1: The first number.
        number_2: The second number.

    Returns:
        int or float: The sum of a and b.
    """
    return number_1 + number_2


def test_add_multiple_inputs():
    ''' An initial attempt that uses a for loop within the test (avoid this).
    '''
    test_data = [(1, 2, 3),
                 (-1, -1, -2),
                 (5, 0, 5)]
    for num_1, num_2, expected in test_data:
        assert add(number_1=num_1, number_2=num_2) == expected

In [ ]:
!pytest -vs test_even.py

In [ ]:
%%writefile test_add_numbers.py
import pytest


def add(number_1: int|float, number_2: int|float) -> float:
    """
    Returns the sum of two numbers.

    Parameters:
        number_1: The first number.
        number_2: The second number.

    Returns:
        int or float: The sum of a and b.
    """
    return number_1 + number_2


@pytest.mark.parametrize("num_1, num_2, expected", [(1, 2, 3),        # Test Case 1: Positive numbers
                                                    (-1, -1, -2),     # Test Case 2: Negative numbers
                                                    (5, 0, 5),        # Test Case 3: Testing with zero
                                                    (100, -50, 50)])  # Test Case 4: Positive and negative

def test_add(num_1, num_2, expected):
    assert add(number_1=num_1, number_2=num_2) == expected

In [ ]:
!pytest -vs test_add_numbers.py

In [ ]:
%rm test_add_numbers.py